# **The fifth in-class-exercise (40 points in total, 11/11/2021)**

(20 points) The purpose of the question is to practice different machine learning algorithms for text classification as well as the performance evaluation. In addition, you are requried to conduct *10 fold cross validation (https://scikit-learn.org/stable/modules/cross_validation.html)* in the training. 

The dataset can be download from here: https://github.com/unt-iialab/info5731_spring2021/blob/main/class_exercises/exercise09_datacollection.zip. The dataset contains two files train data and test data for sentiment analysis in IMDB review, it has two categories: 1 represents positive and 0 represents negative. You need to split the training data into training and validate data (80% for training and 20% for validation, https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6) and perform 10 fold cross validation while training the classifier. The final trained model was final evaluated on the test data. 

Algorithms:

(1) MultinominalNB

(2) SVM 

(3) KNN 

(4) Decision tree

(5) Random Forest

(6) XGBoost

Evaluation measurement:

(1) Accuracy

(2) Recall

(3) Precison 

(4) F-1 score

In [1]:
import pandas as pd
data = open('stsa-train.txt').read()
labels, text = [], []
for i, line in enumerate(data.split("\n")):
    content = line.split(' ')
    labels.append(content[0])
    text.append(" ".join(content[1:]))
testing_data = open('stsa-test.txt').read()
test_labels, test_text = [], []
for i, line in enumerate(testing_data.split("\n")):
    content = line.split(' ')
    test_labels.append(content[0])
    test_text.append(" ".join(content[1:]))


df = pd.DataFrame (list(zip(text, labels)) , columns = ['Raw Text', 'Labels'])
test_df = pd.DataFrame (list(zip(test_text, test_labels)) , columns = ['Raw Text', 'Labels'])
test_df = test_df.dropna()

In [2]:
import nltk
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> all


       | 
       | Downloading package abc to /home/jovyan/nltk_data...
       |   Unzipping corpora/abc.zip.
       | Downloading package alpino to /home/jovyan/nltk_data...
       |   Unzipping corpora/alpino.zip.
       | Downloading package biocreative_ppi to
       |     /home/jovyan/nltk_data...
       |   Unzipping corpora/biocreative_ppi.zip.
       | Downloading package brown to /home/jovyan/nltk_data...
       |   Unzipping corpora/brown.zip.
       | Downloading package brown_tei to /home/jovyan/nltk_data...
       |   Unzipping corpora/brown_tei.zip.
       | Downloading package cess_cat to /home/jovyan/nltk_data...
       |   Unzipping corpora/cess_cat.zip.
       | Downloading package cess_esp to /home/jovyan/nltk_data...
       |   Unzipping corpora/cess_esp.zip.
       | Downloading package chat80 to /home/jovyan/nltk_data...
       |   Unzipping corpora/chat80.zip.
       | Downloading package city_database to
       |     /home/jovyan/nltk_data...
       |   Unzipping


---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

In [3]:
import re
import nltk
from nltk.corpus import stopwords
from textblob import TextBlob
from nltk.stem import PorterStemmer
from textblob import Word
nltk.download('wordnet')
st = PorterStemmer()
nltk.download('punkt')
stop = stopwords.words('english')
df['Lower Case'] = df['Raw Text'].apply(lambda x: " ".join(x.lower() for x in x.split()))
df['Removal of Punctuation'] = df['Lower Case'].str.replace('[^\w\s]','')
df['Removal of Special Characters'] = df['Removal of Punctuation'].apply(lambda x: ''.join(re.sub(r"[^a-zA-Z0-9]+", ' ', charctr) for charctr in x ))
df['Stopwords Removal'] = df['Removal of Punctuation'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
df['Tokenization'] = df['Stopwords Removal'].apply(lambda x: TextBlob(x).words)
df['After Lemmatization'] = df['Tokenization'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x]))


[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/tmp/ipykernel_41/405654497.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Removal of Punctuation'] = df['Lower Case'].str.replace('[^\w\s]','')


In [4]:
test_df['Lower Case'] = test_df['Raw Text'].apply(lambda x: " ".join(x.lower() for x in x.split()))
test_df['Removal of Punctuation'] = test_df['Lower Case'].str.replace('[^\w\s]','')
test_df['Removal of Special Characters'] = test_df['Removal of Punctuation'].apply(lambda x: ''.join(re.sub(r"[^a-zA-Z0-9]+", ' ', charctr) for charctr in x ))
test_df['Stopwords Removal'] = test_df['Removal of Punctuation'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
test_df['Tokenization'] = test_df['Stopwords Removal'].apply(lambda x: TextBlob(x).words)
test_df['After Lemmatization'] = test_df['Tokenization'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x]))

/tmp/ipykernel_41/2737988582.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  test_df['Removal of Punctuation'] = test_df['Lower Case'].str.replace('[^\w\s]','')


In [5]:
# Data Transformations
from sklearn import model_selection, preprocessing, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect = TfidfVectorizer(analyzer='word')
tfidf_vect.fit(df['After Lemmatization'])
x_tfidf =  tfidf_vect.transform(df['After Lemmatization'])
vect_test = TfidfVectorizer(analyzer='word', vocabulary = tfidf_vect.vocabulary_)
vect_test.fit(test_df['After Lemmatization'])
xtest = vect_test.transform(test_df['After Lemmatization'])
test_y = test_df['Labels']

In [6]:
from sklearn import model_selection, preprocessing, naive_bayes, metrics, svm
from sklearn import metrics
import numpy as np
from sklearn.model_selection import cross_val_score, KFold
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(x_tfidf, df['Labels'].values,test_size=0.2)

In [7]:
def get_metrics(predictions, test_data_y):
  accuracy = metrics.accuracy_score(predictions, test_data_y)
  precision = metrics.precision_score(predictions, test_data_y, pos_label='positive', average='micro')
  recall = metrics.recall_score(predictions, test_data_y, pos_label='positive', average='micro')
  f1 = metrics.f1_score(predictions, test_data_y, pos_label='positive', average='micro')
  return accuracy, precision, recall, f1
def cross_validation_score(modelName, x, y):
  scoring = 'accuracy'
  kfold = KFold(10, random_state = 7,shuffle=True)
  cross_val = cross_val_score(modelName, x, y, cv=kfold).mean()
  return cross_val

In [8]:
# Navie Bayes
naive_bayes_model = naive_bayes.MultinomialNB()
naive_bayes_model.fit(train_x, train_y)
naive_bayes_predictions_validation_data = naive_bayes_model.predict(valid_x)
accuracy, precision, recall, f1 = get_metrics(naive_bayes_predictions_validation_data, valid_y)
print("Accuracy is {0}\nPrecision is {1}\nRecall is {2}\nF1 is {3}".format(accuracy, precision, recall, f1))


Accuracy is 0.7776173285198555
Precision is 0.7776173285198555
Recall is 0.7776173285198555
F1 is 0.7776173285198555


/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1298: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn("Note that pos_label (set to %r) is ignored when "
/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1298: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn("Note that pos_label (set to %r) is ignored when "
/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1298: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn("Note that pos_label (set to %r) is ignored when "


In [9]:
# Cross Validation Score
nb_scores_mean = cross_validation_score(naive_bayes_model, valid_x, valid_y)
print('Navie Bayes Cross Validation Score is {0}'.format(nb_scores_mean))

Navie Bayes Cross Validation Score is 0.71049942654572


In [10]:
# Testing metrics
naive_bayes_predictions_test_data = naive_bayes_model.predict(xtest)
accuracy, precision, recall, f1 = get_metrics(naive_bayes_predictions_test_data, test_y)
print("Accuracy is {0}\nPrecision is {1}\nRecall is {2}\nF1 is {3}".format(accuracy, precision, recall, f1))


Accuracy is 0.7941822173435785
Precision is 0.7941822173435785
Recall is 0.7941822173435785
F1 is 0.7941822173435786


/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1298: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn("Note that pos_label (set to %r) is ignored when "
/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1298: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn("Note that pos_label (set to %r) is ignored when "
/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1298: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn("Note that pos_label (set to %r) is ignored when "


In [11]:
# Cross Validation Score - Testing
nb_test_scores_mean = cross_validation_score(naive_bayes_model, xtest, test_y)
print('Navie Bayes Cross Validation Score is {0}'.format(nb_test_scores_mean))

Navie Bayes Cross Validation Score is 0.7371044256290158


In [12]:
# SVM - Validation Metrics

In [13]:
svm_model = svm.SVC()
svm_model.fit(train_x, train_y)
svm_predictions_validation_data = svm_model.predict(valid_x)
accuracy, precision, recall, f1 = get_metrics(svm_predictions_validation_data, valid_y)
print("Accuracy is {0}\nPrecision is {1}\nRecall is {2}\nF1 is {3}".format(accuracy, precision, recall, f1))

Accuracy is 0.768231046931408
Precision is 0.768231046931408
Recall is 0.768231046931408
F1 is 0.768231046931408


/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1298: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn("Note that pos_label (set to %r) is ignored when "
/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1298: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn("Note that pos_label (set to %r) is ignored when "
/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1298: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn("Note that pos_label (set to %r) is ignored when "


In [14]:
# Cross Validation Score
svm_scores_mean = cross_validation_score(svm_model, valid_x, valid_y)
print('SVM Cross Validation Score is {0}'.format(svm_scores_mean))

SVM Cross Validation Score is 0.7061411740173079


In [15]:
# Testing Metrics
svm_predictions_test_data = svm_model.predict(xtest)
accuracy, precision, recall, f1 = get_metrics(svm_predictions_test_data, test_y)
print("Accuracy is {0}\nPrecision is {1}\nRecall is {2}\nF1 is {3}".format(accuracy, precision, recall, f1))

Accuracy is 0.7810098792535675
Precision is 0.7810098792535675
Recall is 0.7810098792535675
F1 is 0.7810098792535675


/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1298: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn("Note that pos_label (set to %r) is ignored when "
/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1298: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn("Note that pos_label (set to %r) is ignored when "
/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1298: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn("Note that pos_label (set to %r) is ignored when "


In [16]:
# Cross Validation Score - Testing
svm_test_scores_mean = cross_validation_score(svm_model, xtest, test_y)
print('SVM Cross Validation Score is {0}'.format(svm_test_scores_mean))

SVM Cross Validation Score is 0.7239086050561461


In [17]:
# KNN - Validation Metrics
from sklearn.neighbors import KNeighborsClassifier
knn_model = KNeighborsClassifier(n_neighbors = 15)
knn_model.fit(train_x, train_y)
knn_predictions_valid_data = knn_model.predict(valid_x)
accuracy, precision, recall, f1 = get_metrics(knn_predictions_valid_data, valid_y)
print("Accuracy is {0}\nPrecision is {1}\nRecall is {2}\nF1 is {3}".format(accuracy, precision, recall, f1))


Accuracy is 0.7292418772563177
Precision is 0.7292418772563177
Recall is 0.7292418772563177
F1 is 0.7292418772563178


/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1298: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn("Note that pos_label (set to %r) is ignored when "
/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1298: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn("Note that pos_label (set to %r) is ignored when "
/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1298: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn("Note that pos_label (set to %r) is ignored when "


In [18]:
# Cross Validation Score
knn_scores_mean = cross_validation_score(knn_model, valid_x, valid_y)
print('KNN Cross Validation Score is {0}'.format(knn_scores_mean))

KNN Cross Validation Score is 0.6779637159837347


In [19]:
# Testing Metrics
knn_predictions_test_data = knn_model.predict(xtest)
accuracy, precision, recall, f1 = get_metrics(knn_predictions_test_data, test_y)
print("Accuracy is {0}\nPrecision is {1}\nRecall is {2}\nF1 is {3}".format(accuracy, precision, recall, f1))


Accuracy is 0.7316136114160263
Precision is 0.7316136114160263
Recall is 0.7316136114160263
F1 is 0.7316136114160263


/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1298: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn("Note that pos_label (set to %r) is ignored when "
/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1298: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn("Note that pos_label (set to %r) is ignored when "
/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1298: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn("Note that pos_label (set to %r) is ignored when "


In [20]:
# Cross Validation Score - Testing
knn_test_scores_mean = cross_validation_score(knn_model, xtest, test_y)
print('KNN Cross Validation Score is {0}'.format(knn_test_scores_mean))


KNN Cross Validation Score is 0.6361376328589443


In [21]:
# Decision Tree - Validation Metrics
from sklearn.tree import DecisionTreeClassifier
dt_model = DecisionTreeClassifier()
dt_model.fit(train_x, train_y)
dt_predictions_valid_data = dt_model.predict(valid_x)
accuracy, precision, recall, f1 = get_metrics(dt_predictions_valid_data, valid_y)
print("Accuracy is {0}\nPrecision is {1}\nRecall is {2}\nF1 is {3}".format(accuracy, precision, recall, f1))


Accuracy is 0.6389891696750902
Precision is 0.6389891696750902
Recall is 0.6389891696750902
F1 is 0.6389891696750902


/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1298: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn("Note that pos_label (set to %r) is ignored when "
/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1298: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn("Note that pos_label (set to %r) is ignored when "
/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1298: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn("Note that pos_label (set to %r) is ignored when "


In [22]:
# Cross Validation Score
dt_scores_mean = cross_validation_score(dt_model, valid_x, valid_y)
print('Decision Tree Cross Validation Score is {0}'.format(dt_scores_mean))


Decision Tree Cross Validation Score is 0.5848295276822021


In [23]:
# Testing Metrics
dt_predictions_test_data = dt_model.predict(xtest)
accuracy, precision, recall, f1 = get_metrics(dt_predictions_test_data, test_y)
print("Accuracy is {0}\nPrecision is {1}\nRecall is {2}\nF1 is {3}".format(accuracy, precision, recall, f1))

Accuracy is 0.6476399560922064
Precision is 0.6476399560922064
Recall is 0.6476399560922064
F1 is 0.6476399560922064


/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1298: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn("Note that pos_label (set to %r) is ignored when "
/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1298: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn("Note that pos_label (set to %r) is ignored when "
/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1298: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn("Note that pos_label (set to %r) is ignored when "


In [24]:
# Cross Validation Score - Testing
dt_test_scores_mean = cross_validation_score(dt_model, xtest, test_y)
print('Decision Tree Cross Validation Score is {0}'.format(dt_test_scores_mean))


Decision Tree Cross Validation Score is 0.6212514261694588


In [25]:
# Random Forest - Validation Metrics
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier()
rf_model.fit(train_x, train_y)
rf_predictions_valid_data = rf_model.predict(valid_x)
accuracy, precision, recall, f1 = get_metrics(rf_predictions_valid_data, valid_y)
print("Accuracy is {0}\nPrecision is {1}\nRecall is {2}\nF1 is {3}".format(accuracy, precision, recall, f1))


Accuracy is 0.71985559566787
Precision is 0.71985559566787
Recall is 0.71985559566787
F1 is 0.71985559566787


/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1298: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn("Note that pos_label (set to %r) is ignored when "
/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1298: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn("Note that pos_label (set to %r) is ignored when "
/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1298: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn("Note that pos_label (set to %r) is ignored when "


In [26]:
# Cross Validation Score
rf_scores_mean = cross_validation_score(rf_model, valid_x, valid_y)
print('Random Forest Cross Validation Score is {0}'.format(rf_scores_mean))


Random Forest Cross Validation Score is 0.6310499426545719


In [27]:
# Testing Metrics
rf_predictions_test_data = rf_model.predict(xtest)
accuracy, precision, recall, f1 = get_metrics(rf_predictions_test_data, test_y)
print("Accuracy is {0}\nPrecision is {1}\nRecall is {2}\nF1 is {3}".format(accuracy, precision, recall, f1))


Accuracy is 0.7327113062568605
Precision is 0.7327113062568605
Recall is 0.7327113062568605
F1 is 0.7327113062568605


/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1298: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn("Note that pos_label (set to %r) is ignored when "
/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1298: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn("Note that pos_label (set to %r) is ignored when "
/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1298: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn("Note that pos_label (set to %r) is ignored when "


In [28]:
# Cross Validation Score - Testing
rf_test_scores_mean = cross_validation_score(rf_model, xtest, test_y)
print('Random Forest Cross Validation Score is {0}'.format(rf_test_scores_mean))


Random Forest Cross Validation Score is 0.6635471086290758


In [30]:
!pip install xgboost

     |████████████████████████████████| 173.5 MB 285 kB/s  eta 0:00:01  |▋                               | 3.5 MB 7.4 MB/s eta 0:00:23     |█████                           | 26.7 MB 18.6 MB/s eta 0:00:08     |█████████▉                      | 53.5 MB 14.0 MB/s eta 0:00:09     |███████████████                 | 81.6 MB 17.3 MB/s eta 0:00:06     |███████████████▎                | 82.6 MB 17.3 MB/s eta 0:00:06     |██████████████████████▊         | 123.3 MB 19.8 MB/s eta 0:00:03     |███████████████████████████████▉| 172.5 MB 19.8 MB/s eta 0:00:01


In [31]:
# XG Boost - Validation Metrics

from xgboost import XGBClassifier
xg_model = XGBClassifier()
xg_model.fit(train_x, train_y)
xg_predictions_valid_data = xg_model.predict(valid_x)
accuracy, precision, recall, f1 = get_metrics(xg_predictions_valid_data, valid_y)
print("Accuracy is {0}\nPrecision is {1}\nRecall is {2}\nF1 is {3}".format(accuracy, precision, recall, f1))


/opt/conda/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[02:45:39] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy is 0.688086642599278
Precision is 0.688086642599278
Recall is 0.688086642599278
F1 is 0.688086642599278


/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1298: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn("Note that pos_label (set to %r) is ignored when "
/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1298: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn("Note that pos_label (set to %r) is ignored when "
/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1298: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn("Note that pos_label (set to %r) is ignored when "


In [32]:
# Cross Validation Score
xg_scores_mean = cross_validation_score(XGBClassifier(), valid_x, valid_y)
print('XG Boost Cross Validation Score is {0}'.format(xg_scores_mean))


/opt/conda/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[02:46:19] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/opt/conda/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[02:46:19] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/opt/conda/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

[02:46:22] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/opt/conda/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[02:46:22] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/opt/conda/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[02:46:23] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XG Boost Cross Validation Score is 0.6080022938171202


In [33]:
# Testing Metrics
xg_predictions_test_data = xg_model.predict(xtest)
accuracy, precision, recall, f1 = get_metrics(xg_predictions_test_data, test_y)
print("Accuracy is {0}\nPrecision is {1}\nRecall is {2}\nF1 is {3}".format(accuracy, precision, recall, f1))

Accuracy is 0.6849615806805708
Precision is 0.6849615806805708
Recall is 0.6849615806805708
F1 is 0.6849615806805708


/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1298: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn("Note that pos_label (set to %r) is ignored when "
/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1298: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn("Note that pos_label (set to %r) is ignored when "
/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1298: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn("Note that pos_label (set to %r) is ignored when "


In [34]:
# Cross Validation Score - Testing
xg_test_scores_mean = cross_validation_score(XGBClassifier(), xtest, test_y)
print('XG Boost Cross Validation Score is {0}'.format(xg_test_scores_mean))

/opt/conda/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[02:46:31] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/opt/conda/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[02:46:31] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/opt/conda/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

[02:46:35] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/opt/conda/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[02:46:36] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/opt/conda/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[02:46:37] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XG Boost Cross Validation Score is 0.6454392601933584


Accuracies of the models are as follows when use test data -

Navie Bayes - 78.14%

SVM - 78.30%

KNN - 72.92%

Decision Tree - 67.14%

Random Forest - 73.14%

XG Boost - 62.92%

(20 points) The purpose of the question is to practice different machine learning algorithms for text clustering
Please downlad the dataset by using the following link.  https://www.kaggle.com/PromptCloudHQ/amazon-reviews-unlocked-mobile-phones
(You can also use different text data which you want)

Apply the listed clustering methods to the dataset:

K means, 
DBSCAN,
Hierarchical clustering. 

You can refer to of the codes from  the follwing link below. 
https://www.kaggle.com/karthik3890/text-clustering 

In [7]:
#Write your code here.
import pandas as pd
Cluster_Data = pd.read_csv("Amazon_Unlocked_Mobile.csv",nrows=30000)

Cluster_Data.dropna(inplace=True)
Cluster_Data.reset_index(inplace = True)

In [8]:
Cluster_Data.shape

(22340, 7)

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
import re
import nltk
# nltk.download('stopwords')
# nltk.download('wordnet')
# nltk.download('all')
Cluster_Data.columns


Index(['index', 'Product Name', 'Brand Name', 'Price', 'Rating', 'Reviews',
       'Review Votes'],
      dtype='object')

In [10]:
from nltk.corpus import stopwords
stop = set(stopwords.words('english'))
from nltk.corpus import stopwords
def remove_stopword(word):
    return word not in words

from nltk.stem import WordNetLemmatizer
Lemma = WordNetLemmatizer()

from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

Cluster_Data['NewReviews'] = Cluster_Data['Reviews'].str.lower().str.split()
Cluster_Data['NewReviews'] = Cluster_Data['NewReviews'].apply(lambda x : [item for item in x if item not in stop])


In [11]:
Cluster_Data['Cleaned_reviews'] = [''.join([WordNetLemmatizer().lemmatize(re.sub('[^A-Za-z]', ' ', line))
for line in lists]).strip() for lists in Cluster_Data['NewReviews']] 

In [12]:
Cluster_Data.shape

(22340, 9)

In [13]:
Cluster_Data.columns

Index(['index', 'Product Name', 'Brand Name', 'Price', 'Rating', 'Reviews',
       'Review Votes', 'NewReviews', 'Cleaned_reviews'],
      dtype='object')

In [14]:
vectorizer = TfidfVectorizer(max_df=0.5,max_features=10000,min_df=10,stop_words='english',use_idf=True)
# Cluster_Data["Vectors"] = vectorizer.fit_transform(Cluster_Data['Cleaned_reviews'].str.upper())
X = vectorizer.fit_transform(Cluster_Data['Cleaned_reviews'].str.upper())

In [15]:
from sklearn.cluster import KMeans
km = KMeans(n_clusters=5,init='k-means++',max_iter=200,n_init=1)
km.fit(X)
terms = vectorizer.get_feature_names()
order_centroids = km.cluster_centers_.argsort()[:,::-1]
for i in range(5):
    print("cluster %d:" %i,end='')
    for ind in order_centroids[i,:10]:
        print(' %s' % terms[ind],end='')
    print()    


cluster 0: mobile fine cost happyphone far unlocked phone times bought good
cluster 1: good greatphone great excellent loveit perfect thankyou lovephone love phone
cluster 2: nice thanks phonedescribed boughtphone new price perfectcondition workwell likenew yes
cluster 3: iphone gs new loveit plus faster gbphone storage cost unlocked
cluster 4: workgreat thanks loveit fastshipping lovephone workgood greatphone goodcondition cametime productdescribed


In [16]:
Cluster_Data["KMEANS"] = km.labels_
Cluster_Data["KMEANS"].value_counts()

1    21261
4      532
3      316
2      127
0      104
Name: KMEANS, dtype: int64

In [17]:
from sklearn.cluster import DBSCAN

In [18]:
dbscan = DBSCAN(eps=0.123, min_samples = 2)
clusters = dbscan.fit_predict(X)

In [19]:
Cluster_Data["DBSCAN"] = dbscan.labels_
Cluster_Data["DBSCAN"].value_counts()

 0       10139
-1        1142
 8         581
 31        267
 30        241
         ...  
 1041        2
 1042        2
 1043        2
 1044        2
 582         2
Name: DBSCAN, Length: 1509, dtype: int64

In [20]:
from sklearn.cluster import AgglomerativeClustering
model = AgglomerativeClustering(n_clusters=5, affinity='euclidean')
model.fit(X.toarray())

AgglomerativeClustering(n_clusters=5)

In [21]:
Cluster_Data["Hierarchical clustering"] = model.labels_
Cluster_Data["Hierarchical clustering"].value_counts()

0    21016
3      582
2      267
4      243
1      232
Name: Hierarchical clustering, dtype: int64

In [22]:
Cluster_Data

,index,Product Name,Brand Name,Price,Rating,Reviews,Review Votes,NewReviews,Cleaned_reviews,KMEANS,DBSCAN,Hierarchical clustering
0,0,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,I feel so LUCKY to have found this used (phone...,1.0,"[feel, lucky, found, used, (phone, us, &, used...",feelluckyfoundused phoneu usedhardall phoneli...,1,0,0
1,1,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,"nice phone, nice up grade from my pantach revu...",0.0,"[nice, phone,, nice, grade, pantach, revue., c...",nicephone nicegradepantachrevue cleanseteasyse...,1,1,0
2,2,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,Very pleased,0.0,[pleased],pleased,1,2,0
3,3,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,It works good but it goes slow sometimes but i...,0.0,"[works, good, goes, slow, sometimes, good, pho...",workgoodgoslowsometimesgoodphonelove,1,0,0
4,4,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,Great phone to replace my lost phone. The only...,0.0,"[great, phone, replace, lost, phone., thing, v...",greatphonereplacelostphone thingvolumebuttonwo...,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
22335,29995,Apple iPhone 5c 32GB (Blue) - AT&T,Apple,274.95,5,The description said it was in 'very good' con...,0.0,"[description, said, 'very, good', condition,, ...",descriptionsaid verygood condition actuallyrec...,1,1387,0
22336,29996,Apple iPhone 5c 32GB (Blue) - AT&T,Apple,274.95,5,The phone was just like described . Not a sing...,0.0,"[phone, like, described, ., single, scratch, s...",phonelikedescribed singlescratchscreenshippedw...,1,0,0
22337,29997,Apple iPhone 5c 32GB (Blue) - AT&T,Apple,274.95,2,Gave 3 stars because it did not come with a ch...,0.0,"[gave, 3, stars, come, charger, box., battery,...",gave starcomechargerbox batteryalreadyhalfwayg...,1,234,0
22338,29998,Apple iPhone 5c 32GB (Blue) - AT&T,Apple,274.95,5,phone looks new! Works great!,0.0,"[phone, looks, new!, works, great!]",phonelooknew workgreat,4,57,0
